In [1]:
using CSV, DataFrames, GLM, BenchmarkTools, RegressionTables, FixedEffectModels
cd("/Users/junwong/Dropbox/Second Year/Dingel - Trade/Assignments")

In [2]:
df = DataFrame(CSV.File("data/Detroit.csv"))
transform!(df, [:flows, :distance_Google_miles, :duration_minutes] .=> 
    (x -> log.(x)) .=> 
    [:flows, :distance_Google_miles, :duration_minutes]);
df = df[df.flows .> -Inf, :];

In [3]:
# time this function
run_regressions = function(data) 
    r1 = @timed reg(data, @formula(flows ~ distance_Google_miles + fe(work_ID) + fe(home_ID)), Vcov.robust());
    r2 = @timed reg(data, @formula(flows ~ duration_minutes + fe(work_ID) + fe(home_ID)), Vcov.robust());
    return [r1, r2]
end 

# initialize regressions
run_regressions(df[1:5000,:]);

# actual regressions
results = run_regressions(df);

In [4]:
# make regression table 
dict = Dict("flows"=> "Log(Flows)", "distance_Google_miles" => "log(Distance)", 
    "duration_minutes" => "Log(Duration)", "work_ID" => "Destination FE", 
    "home_ID" => "Origin FE", "__LABEL_CUSTOM_STATISTIC_command__" => "Command", 
    "__LABEL_CUSTOM_STATISTIC_timer__" => "Run Time");

timer = [results[1][2], results[2][2]]
command = ["texttt{FixedEffectModels}", "texttt{FixedEffectModels}"]
customstats = NamedTuple{(:command, :timer)}((command, timer))

regtable(results[1][1], results[2][1], 
    renderSettings = latexOutput("output/table_3_julia.tex"), 
    labels = dict, print_estimator_section = false, 
    custom_statistics = customstats)
